In [ ]:
#Пример Google Sheet который конвертнул в csv: https://docs.google.com/spreadsheets/d/1ixw5YIfXPwAJyse1WAg00vYDjd6wwT3_KG8PFrpBaQ8/edit#gid=0
#Это первый шаг, делать его обазетельно

from bs4 import BeautifulSoup
#from fake_useragent import UserAgent -- если нужно проверять с мобилы
import pandas as pd
import requests
import warnings; warnings.simplefilter('ignore') #убираем предупреждения
import re

#ua = UserAgent()

df = pd.read_csv('url_cmrt.ru.csv')

df['tel'] = df['tel'].astype('str')

for i in range(len(df)):
    if 'http://' not in df.iloc[i, 0]:
        df.iloc[i, 0] = 'http://' + df.iloc[i, 0]
        
    if len(df.loc[i, 'tel']) == 11:
        df.loc[i, 'tel'] = '+' + str(df.loc[i, 'tel'])
    elif len(df.loc[i, 'tel']) == 10:
        df.loc[i, 'tel'] = '+7' + str(df.loc[i, 'tel'])
        
    url = df.loc[i, 'Url']
    soup = BeautifulSoup(requests.get(url, headers={'Cache-Control': 'no-cache'}).text)
    links = soup.find_all('a', href=re.compile("tel:"))
    
    link_temporary = []
    
    for link in links:
        l = link['href']
        link_temporary.append(l[4:])
    
    for b in range(len(link_temporary)):
        for a in link_temporary:
            here = 'tel-' + str(b)
            df.loc[i, here] = a
            df[here] = df[here].astype('str')
    
            if len(df.loc[i, here]) > 12:
                df.loc[i, here] = df.loc[i, here].replace(" ", "")        
                df.loc[i, here] = df.loc[i, here].replace("\u200b", "")
            df.loc[i, 'NoError-' + here] = (df.loc[i, 'tel'] == df.loc[i, here])

df.to_csv('cmrt-out.csv', sep=',')

df

In [ ]:
# Второй шаг — показывает только значения где есть ошибки. Первый шаг сделать обязательно.

df_column = df.columns.get_values()
column_result = [s for s in df_column if 'NoError-tel' in s]

list_errors = []

for x in column_result:
    df_temp = df.loc[~df[x].isin([True])].index.values.tolist()
    for y in df_temp:
        list_errors.append(y)
    
output = []
for x in list_errors:
    if x not in output:
        output.append(x)

if len(output) == 0:
    print('Ошибок нет')
else:
    print('Ошибки находятся в строках: ' + str(output))
    display(df.loc[df.index.isin(output)])